In [14]:
import torch
from torch import nn
from torch.utils.data.dataloader import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Download training data from open datasets
training_data = datasets.FashionMNIST(root='data', train=True, download=True, transform=ToTensor())

# Download test data from open datasets
test_data = datasets.FashionMNIST(root='data', train=False, download=True, transform=ToTensor())


In [15]:
batch_size = 64

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f'Shape of X [N, C, H, W]: {X.shape}')
    print(f'Shape of y : {y.shape}, {y.dtype}')
    break


Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y : torch.Size([64]), torch.int64


In [16]:
device = (
    'cuda'
    if torch.cuda.is_available()
    else 'mps'
    if torch.backends.mps.is_available()
    else 'cpu'
)
print(f'Using {device} device')


Using cpu device


In [17]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [18]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [19]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f'loss :  {loss:>7f} [{current:>5d} / {size:>5d}]')

In [20]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f'Test error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n')

In [21]:
epochs = 5
for t in range(epochs):
    print(f'Epoch { t + 1 } \n ---------------------------------')
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print('Done!')

Epoch 1 
 ---------------------------------
loss :  2.311991 [   64 / 60000]
loss :  2.293175 [ 6464 / 60000]
loss :  2.270723 [12864 / 60000]
loss :  2.257314 [19264 / 60000]
loss :  2.253881 [25664 / 60000]
loss :  2.211052 [32064 / 60000]
loss :  2.228619 [38464 / 60000]
loss :  2.192634 [44864 / 60000]
loss :  2.187249 [51264 / 60000]
loss :  2.152629 [57664 / 60000]
Test error: 
 Accuracy: 41.8%, Avg loss: 2.146511 

Epoch 2 
 ---------------------------------
loss :  2.157106 [   64 / 60000]
loss :  2.147255 [ 6464 / 60000]
loss :  2.086171 [12864 / 60000]
loss :  2.102957 [19264 / 60000]
loss :  2.063813 [25664 / 60000]
loss :  1.984619 [32064 / 60000]
loss :  2.029902 [38464 / 60000]
loss :  1.946095 [44864 / 60000]
loss :  1.954410 [51264 / 60000]
loss :  1.877684 [57664 / 60000]
Test error: 
 Accuracy: 59.4%, Avg loss: 1.875569 

Epoch 3 
 ---------------------------------
loss :  1.904025 [   64 / 60000]
loss :  1.877713 [ 6464 / 60000]
loss :  1.757646 [12864 / 60000]
loss 

In [22]:
torch.save(model.state_dict(), '01.model.pth')
print('Saved PyTorch model state to 01.model.pth')

Saved PyTorch model state to 01.model.pth


In [23]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load('01.model.pth'))

<All keys matched successfully>

In [24]:
classes = [
    'T-shirt/top',
    'Trouser',
    'Pullover',
    'Dress',
    'Coat',
    'Sandal',
    'Shirt',
    'Sneaker',
    'Bag',
    'Ankle boot'
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
